In [1]:
"""
Baseline for CIL project on road segmentation.
This simple baseline consits of a CNN with two convolutional+pooling layers with a soft-max loss
"""

import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
from PIL import Image

import code

import tensorflow.python.platform

import numpy as np
import tensorflow as tf

import cil_helper as cil

In [2]:
from skimage.feature import greycomatrix, greycoprops

In [3]:
try:
    tf.app.flags.DEFINE_string('log_dir', '/tmp/tensorflow_lukas',
                           """Directory where to write event logs """
                           """and checkpoint.""")
except: 
    print(tf.app.flags.FLAGS.log_dir)
    
FLAGS = tf.app.flags.FLAGS

In [4]:
TRAINING_SIZE = 20
VALIDATION_SIZE = 5  # Size of the validation set.
TEST_SIZE = 50
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 2000 # 64
NUM_EPOCHS = 2
RESTORE_MODEL = False # If True, restore existing model instead of training a new one
RECORDING_STEP = 100

# Set image patch size in pixels
# IMG_PATCH_SIZE should be a multiple of 4
# image size should be an integer multiple of this number!
IMG_PATCH_SIZE = 1

In [12]:
NUM_CHANNELS = 3

In [13]:
# Extract label images
def extract_labels(filename, num_images):
    """Extract the labels into a 1-hot matrix [image index, label index]."""
    gt_imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            gt_imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(gt_imgs)
    IMG_WIDTH = gt_imgs[0].shape[0]
    IMG_HEIGHT = gt_imgs[0].shape[1]
    IMG_LENGTH = IMG_WIDTH*IMG_HEIGHT
    
    labels = np.reshape(gt_imgs, [IMG_LENGTH*num_images])

    # Convert to dense 1-hot representation.
    return labels.astype(np.float32)

def extract_data(filename, num_images):
    """Extract the images into a 2D tensor [image index, channels].
    Values are rescaled from [0, 255] down to [-0.5, 0.5].
    """
    imgs = []
    for i in range(1, num_images+1):
        imageid = "satImage_%.3d" % i
        image_filename = filename + imageid + ".png"
        if os.path.isfile(image_filename):
            #print ('Loading ' + image_filename)
            img = mpimg.imread(image_filename)
            IMG_LENGTH = img.shape[0]*img.shape[1]
            img = np.reshape(img, [IMG_LENGTH, NUM_CHANNELS])
            imgs.append(img)
        else:
            print ('File ' + image_filename + ' does not exist')

    num_images = len(imgs)
    IMG_WIDTH = imgs[0].shape[0]
    IMG_HEIGHT = imgs[0].shape[1]
    IMG_LENGTH = IMG_WIDTH*IMG_HEIGHT
    
    print(num_images*IMG_LENGTH)
    print(NUM_CHANNELS)
    imgs = np.asarray(imgs)
    print(imgs.shape)
    
    shape = imgs.shape
    data = np.reshape(imgs, [shape[0]*shape[1], shape[2]])
    
    #return None

    #img_patches = [cil.img_crop(imgs[i], IMG_PATCH_SIZE, IMG_PATCH_SIZE) for i in range(num_images)]
    #data = [img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))]

    #data = np.reshape(imgs, [IMG_LENGTH, NUM_CHANNELS, num_images])
    #data = tf.reshape(data, [data.get_shape()[0], NUM_CHANNELS])
    #np.reshape(data, (train_data.shape[0], NUM_CHANNELS))
    return data #np.asarray(data)

In [14]:
# Assign a label to a patch v
def value_to_class(v):
    foreground_threshold = 0.5 # percentage of pixels > 1 required to assign a foreground label to a patch
    #ret#urn [1-v , v]
    df = np.sum(v)
    if df > foreground_threshold:
       return [0, 1]
    else:
       return [1, 0]

In [15]:
data_dir = 'training/'
train_data_filename = data_dir + 'images/'
train_labels_filename = data_dir + 'groundtruth/' 

# Extract it into numpy arrays.
train_data = extract_data(train_data_filename, TRAINING_SIZE)
train_labels = extract_labels(train_labels_filename, TRAINING_SIZE)

num_epochs = NUM_EPOCHS

9600000
3
(20, 160000, 3)


In [16]:
NUM_CHANNELS = 3
print(train_data.shape)
print(train_labels.shape)

(3200000, 3)
(3200000,)


In [17]:
c0 = 0
c1 = 0
th = 0.25
for i in range(len(train_labels)):
    if train_labels[i] > th:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print ('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 586996 c1 = 2613004


In [18]:
print ('Balancing training data...')
min_c = min(c0, c1)
print(min_c)
idx0 = [i for i, j in enumerate(train_labels) if j > th]
idx1 = [i for i, j in enumerate(train_labels) if j <= th]
new_indices = idx0[0:min_c] + idx1[0:min_c]
print(len(new_indices))
print(train_data.shape)
train_data = train_data[new_indices,:]
train_labels = train_labels[new_indices]

Balancing training data...
586996
1173992
(3200000, 3)


In [19]:
train_size = train_labels.shape[0]
print(train_size)

1173992


In [20]:
# This is where training samples and labels are fed to the graph.
# These placeholder nodes will be fed a batch of training data at each
# training step using the {feed_dict} argument to the Run() call below.
train_data_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE, NUM_CHANNELS),
    name='train_data')
train_labels_node = tf.placeholder(
    tf.float32,
    shape=(BATCH_SIZE),
    name='train_labels')
train_all_data_node = tf.constant(train_data)

In [22]:
# The variables below hold all the trainable weights. They are passed an
# initial value which will be assigned when when we call:
# {tf.initialize_all_variables().run()}
fc1_weights = tf.Variable(  # fully connected, depth 512.
    tf.truncated_normal([NUM_CHANNELS, 2*NUM_CHANNELS],
                        stddev=0.1,
                        seed=SEED))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[2*NUM_CHANNELS]))
fc2_weights = tf.Variable(
    tf.truncated_normal([2*NUM_CHANNELS, 1],
                        stddev=0.1,
                        seed=SEED))
fc2_biases = tf.Variable(tf.constant(0.1, shape=[1]))

In [23]:
# We will replicate the model structure for the training subgraph, as well
# as the evaluation subgraphs, while sharing the trainable parameters.
def model(data, train=False):
    """The Model definition."""

    fc1 = tf.nn.relu(tf.matmul(data, fc1_weights) + fc1_biases)
    if train:
        fc1 = tf.nn.dropout(fc1, 0.6, seed=SEED)
    fc2 = tf.matmul(fc1, fc2_weights) + fc2_biases

    return fc2

In [24]:
# Training computation: logits + cross-entropy loss.
logits = model(train_data_node, True) # BATCH_SIZE*NUM_LABELS
# print 'logits = ' + str(logits.get_shape()) + ' train_labels_node = ' + str(train_labels_node.get_shape())
#loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#    logits, train_labels_node))
loss = tf.reduce_mean(tf.square(logits - train_labels_node))
loss_summary = tf.scalar_summary('loss', loss)

In [27]:
all_params_node = [fc1_weights, fc1_biases, fc2_weights, fc2_biases]
all_params_names = ['fc1_weights', 'fc1_biases', 'fc2_weights', 'fc2_biases']
all_grads_node = tf.gradients(loss, all_params_node)
all_grad_norms_node = []
for i in range(0, len(all_grads_node)):
    norm_grad_i = tf.global_norm([all_grads_node[i]])
    all_grad_norms_node.append(norm_grad_i)
    all_params_summary = tf.scalar_summary(all_params_names[i], norm_grad_i)

In [28]:
# L2 regularization for the fully connected parameters.
regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))
# Add the regularization term to the loss.
loss += 5e-4 * regularizers

In [29]:
# Optimizer: set up a variable that's incremented once per batch and
# controls the learning rate decay.
batch = tf.Variable(0)
# Decay once per epoch, using an exponential schedule starting at 0.01.
learning_rate = tf.train.exponential_decay(
    0.015,                # Base learning rate.
    batch * BATCH_SIZE,  # Current index into the dataset.
    train_size,          # Decay step.
    0.95,                # Decay rate.
    staircase=True)
learning_rate_summary = tf.scalar_summary('learning_rate', learning_rate)

In [30]:
# Use simple momentum for the optimization.
#optimizer = tf.train.MomentumOptimizer(learning_rate, 0.0).minimize(loss, global_step=batch)

# try adam
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=batch)

In [31]:
# Predictions for the minibatch, validation set and test set.
train_prediction = tf.nn.softmax(logits)
# We'll compute them only once in a while by calling their {eval()} method.
train_all_prediction = tf.nn.softmax(model(train_all_data_node))

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

In [32]:
s = tf.InteractiveSession()

In [33]:
# Get prediction overlaid on the original image for given input file
def get_prediction_with_overlay(filename, image_idx):

    imageid = "satImage_%.3d" % image_idx
    image_filename = filename + imageid + ".png"
    img = mpimg.imread(image_filename)

    img_prediction = get_prediction(img)
    oimg = cil.make_img_overlay(img, img_prediction)

    return oimg

# Get a concatenation of the prediction and groundtruth for given input file
def get_prediction_with_groundtruth(filename, image_idx):

    imageid = "satImage_%.3d" % image_idx
    image_filename = filename + imageid + ".png"
    img = mpimg.imread(image_filename)

    img_prediction = get_prediction(img)
    cimg = cil.concatenate_images(img, img_prediction)

    return cimg

# Get prediction for given input image 
def get_prediction(img):
    data = np.asarray(cil.img_crop(img, IMG_PATCH_SIZE, IMG_PATCH_SIZE))
    data_node = tf.constant(data)
    output = tf.nn.softmax(model(data_node))
    output_prediction = s.run(output)
    img_prediction = cil.label_to_img(img.shape[0], img.shape[1], IMG_PATCH_SIZE, IMG_PATCH_SIZE, output_prediction)

    return img_prediction

In [39]:
def error_rate(predictions, labels):
    """Return the error rate based on dense predictions and 1-hot labels."""
    print(predictions.shape)
    return np.sum(np.subtract(predictions, labels))

In [40]:
if RESTORE_MODEL:
    # Restore variables from disk.
    saver.restore(FLAGS.train_dir + "/model.ckpt")
    print("Model restored.")

In [41]:
# Run all the initializers to prepare the trainable parameters.
tf.initialize_all_variables().run()

# Build the summary operation based on the TF collection of Summaries.
summary_op = tf.merge_summary([loss_summary, learning_rate_summary, all_params_summary])
summary_writer = tf.train.SummaryWriter(FLAGS.log_dir,
                                                graph=s.graph)
print ('Initialized!')

# Loop through training steps.
print ('Total number of iterations = ' + str(int(num_epochs * train_size / BATCH_SIZE)))

training_indices = range(train_size)

for iepoch in range(num_epochs):

    # Permute training indices
    perm_indices = np.random.permutation(training_indices)

    for step in range (int(train_size / BATCH_SIZE)):

        offset = (step * BATCH_SIZE) % (train_size - BATCH_SIZE)
        batch_indices = perm_indices[offset:(offset + BATCH_SIZE)]

        # Compute the offset of the current minibatch in the data.
        # Note that we could use better randomization across epochs.
        batch_data = train_data[batch_indices, :]
        batch_labels = train_labels[batch_indices]
        
        #print('batch_data.shape = ' + str(batch_data.shape))
        # This dictionary maps the batch data (as a numpy array) to the
        # node in the graph is should be fed to.
        feed_dict = {train_data_node: batch_data, train_labels_node: batch_labels}

        if step % RECORDING_STEP == 0:

            summary_str, _, l, lr, predictions = s.run(
                        [summary_op, optimizer, loss, learning_rate, train_prediction],
                        feed_dict=feed_dict)
            summary_writer.add_summary(summary_str, step)
            summary_writer.flush()
            
            l, lr, all_predictions = s.run(
                        [loss, learning_rate, train_all_prediction],
                        feed_dict=feed_dict)
            # print_predictions(predictions, batch_labels)

            #print('Epoch: ' + str(iepoch))
            #print('Step: ' + str(step))
            print ('Minibatch loss: %.3f, learning rate: %.6f' % (l, lr))
            print ('Minibatch error: %.1f%%' % error_rate(predictions,
                                                                 batch_labels))
            #print('Step: ' + str(step + iepoch*int(train_size / BATCH_SIZE)) + ' error: %.1f%%' % error_rate(all_predictions, train_labels))
            #print(batch_labels)
            sys.stdout.flush()
        else:
            # Run the graph and fetch some of the nodes.
            _, l, lr, predictions = s.run(
                        [optimizer, loss, learning_rate, train_prediction],
                        feed_dict=feed_dict)

# Save the variables to disk.
save_path = saver.save(s, FLAGS.train_dir + "/model.ckpt")
print("Model saved in file: %s" % save_path) 

Initialized!
Total number of iterations = 1173
Minibatch loss: 0.330, learning rate: 0.015000
(2000, 1)
Minibatch error: 2209020.5%
Minibatch loss: 0.210, learning rate: 0.015000
(2000, 1)
Minibatch error: 2180291.5%
Minibatch loss: 0.210, learning rate: 0.015000
(2000, 1)
Minibatch error: 2166383.8%
Minibatch loss: 0.211, learning rate: 0.015000
(2000, 1)
Minibatch error: 2223991.0%
Minibatch loss: 0.211, learning rate: 0.015000
(2000, 1)
Minibatch error: 2109591.2%
Minibatch loss: 0.209, learning rate: 0.015000
(2000, 1)
Minibatch error: 2201654.2%
Minibatch loss: 0.209, learning rate: 0.014250
(2000, 1)
Minibatch error: 2127780.0%
Minibatch loss: 0.211, learning rate: 0.014250
(2000, 1)
Minibatch error: 2206022.8%
Minibatch loss: 0.210, learning rate: 0.014250
(2000, 1)
Minibatch error: 2185088.5%
Minibatch loss: 0.212, learning rate: 0.014250
(2000, 1)
Minibatch error: 2073906.1%
Minibatch loss: 0.211, learning rate: 0.014250
(2000, 1)
Minibatch error: 2168698.0%
Minibatch loss: 0.

In [42]:
print ("Running prediction on training set")
prediction_training_dir = "predictions_training/"
if not os.path.isdir(prediction_training_dir):
    os.mkdir(prediction_training_dir)
for i in range(1, TRAINING_SIZE+1):
    pimg = get_prediction_with_groundtruth(train_data_filename, i)
    Image.fromarray(pimg).save(prediction_training_dir + "prediction_" + str(i) + ".png")
    oimg = get_prediction_with_overlay(train_data_filename, i)
    oimg.save(prediction_training_dir + "overlay_" + str(i) + ".png")  

Running prediction on training set


ValueError: Shape (160000, 1, 1, 3) must have rank 2

In [ ]:
print ("Running prediction on test set")
prediction_test_dir = "predictions_test/"
test_dir = "test/"
if not os.path.isdir(prediction_test_dir):
    os.mkdir(prediction_test_dir)
for i in range(1, TEST_SIZE+1):
    imagename = "test_" + str(i)
    image_filename = test_dir + imagename + "/" + imagename + ".png"
    img = mpimg.imread(image_filename)
    pimg = cil.img_float_to_uint8(get_prediction(img))
    Image.fromarray(pimg).save(prediction_test_dir + "prediction_" + str(i) + ".png")

In [ ]:
import mask_to_submission as mts
mts.save('submission_lukas_10_1.csv')